In [1]:
from selenium import webdriver

options = webdriver.ChromeOptions()
options.headless = True
options.add_argument("window-size=1920x1080")

browser = webdriver.Chrome(options=options)
browser.maximize_window()

url = "https://www.whatismybrowser.com/detect/what-is-my-user-agent"
browser.get(url)

detected_value = browser.find_element_by_id("detected_value")
print(detected_value.text)
browser.quit()

<ipython-input-1-d96cc43e8ccd>:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  detected_value = browser.find_element_by_id("detected_value")


Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/96.0.4664.93 Safari/537.36


- HeadlessChrome : 따로 유저설정을 하지 않은 경우 -> 서버 입장에선 브라우저 접속을 막을 수 있음
- 이를 해결하기 위해 add_arguemnt에 user- agent 값을 추가함

In [2]:
from selenium import webdriver

options = webdriver.ChromeOptions()
options.headless = True
options.add_argument("window-size=1920x1080")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36")

browser = webdriver.Chrome(options=options)
browser.maximize_window()

url = "https://www.whatismybrowser.com/detect/what-is-my-user-agent"
browser.get(url)

detected_value = browser.find_element_by_id("detected_value")
print(detected_value.text)
browser.quit()

<ipython-input-2-80bf56b4a75d>:18: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  detected_value = browser.find_element_by_id("detected_value")


Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36


In [1]:
from selenium import webdriver
import requests
from bs4 import BeautifulSoup

options = webdriver.ChromeOptions()
options.headless = True
options.add_argument("window-size=1920x1080")

browser = webdriver.Chrome(options=options)
browser.maximize_window()

url = "https://play.google.com/store/movies/collection/cluster?clp=0g4XChUKD3RvcHNlbGxpbmdfcGFpZBAHGAQ%3D:S:ANO1ljJvXQM&gsr=ChrSDhcKFQoPdG9wc2VsbGluZ19wYWlkEAcYBA%3D%3D:S:ANO1ljK7jAA&hl=ko&gl=US"
headers = {
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36",
    "Accept-Language":"ko-KR,ko"
          }
res = requests.get(url, headers = headers)
res.raise_for_status()

browser.get(url)

import time
interval = 2 # 2초에 한번씩 스크롤 내림
# 현재 문서 높이를 가져와서 저장
prev_height = browser.execute_script("return document.body.scrollHight")

# 반복 수행
while True:
    # 스크롤을 가장 아래로 내림
    browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    
    # 페이지 로딩 대기
    time.sleep(interval)
    
    # 현재 문서 높이를 가져와서 저장
    curr_height = browser.execute_script("return document.body.scrollHegiht")
    
    if curr_height == prev_height:
        break
    
    prev_height = curr_height
    
print("스크롤 완료")
browser.get_screenshot_as_file("google_movie.png")


soup = BeautifulSoup(browser.page_source, "lxml")

movies = soup.find_all("div", attrs = {"class":"Vpfmgd"})
print(len(movies))

for movie in movies:
    title = movie.find("div", attrs={"class":"WsMG1c nnK0zc"}).get_text()
    
    # 할인 전 가격
    origin_price = movie.find("span", attrs = {"class":"SUZt4c djCuy"})
    if origin_price:
        origin_price = origin_price.get_text()
    else:
        #print(title, "<할인 되지 않은 영화 제외>")
        continue
    
    # 할인 된 가격
    discount_price = movie.find("span", attrs = {"class":"VfPpfd ZdBevf i5DZme"}).get_text()
    
    # 링크
    link = movie.find("a", attrs = {"class":"JC71ub"})["href"]
    link = "https://play.google.com" + link
    
    print(f"제목 : {title}")
    print(f"할인 전 금액 : {origin_price}")
    print(f"할인 된 금액 : {discount_price}")
    print(f"링크 : {link}")
    print("\n")

C:\Users\seoyu\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


스크롤 완료
150
제목 : 듄
할인 전 금액 : US$24.99
할인 된 금액 : US$5.99
링크 : https://play.google.com/store/movies/details/%EB%93%84?id=r9FqbC-kvqs.P


제목 : Last Night in Soho
할인 전 금액 : US$19.99
할인 된 금액 : US$5.99
링크 : https://play.google.com/store/movies/details/Last_Night_in_Soho?id=7OmPbA-0Jaw.P


제목 : Spencer
할인 전 금액 : US$6.99
할인 된 금액 : US$3.99
링크 : https://play.google.com/store/movies/details/Spencer?id=Syv-16fUi2A.P


제목 : 콰이어트 플레이스 2
할인 전 금액 : US$4.99
할인 된 금액 : US$2.99
링크 : https://play.google.com/store/movies/details/%EC%BD%B0%EC%9D%B4%EC%96%B4%ED%8A%B8_%ED%94%8C%EB%A0%88%EC%9D%B4%EC%8A%A4_2?id=vO82xm8l-E4.P


제목 : The Hating Game
할인 전 금액 : US$6.99
할인 된 금액 : US$3.99
링크 : https://play.google.com/store/movies/details/The_Hating_Game?id=z2vIuzDx8Cw.P


제목 : 스네이크 아이즈: 지.아이.조
할인 전 금액 : US$4.99
할인 된 금액 : US$2.99
링크 : https://play.google.com/store/movies/details/%EC%8A%A4%EB%84%A4%EC%9D%B4%ED%81%AC_%EC%95%84%EC%9D%B4%EC%A6%88_%EC%A7%80_%EC%95%84%EC%9D%B4_%EC%A1%B0?id=h23JaCc87xk.P


제목 : Spider-Man Tril